In [1]:
import requests
import time
from bs4 import BeautifulSoup
import json

In [253]:
headers = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36'}
url = 'https://scholar.google.com/citations?hl=en&user=M_BRU-4AAAAJ&view_op=list_works&sortby=pubdate'

In [254]:
response=requests.get(url,headers=headers)

In [255]:
response

<Response [200]>

In [256]:
# this function for the getting inforamtion of the web page
def get_paperinfo(paper_url):
    #download the page
    response=requests.get(url,headers=headers)

    # check successful response
    if response.status_code != 200:
        print('Status code:', response.status_code)
        raise Exception('Failed to fetch web page ')

    #parse using beautiful soup
    paper_doc = BeautifulSoup(response.text,'html.parser')

    return paper_doc

In [257]:
# this function for the extracting information of the tags
def get_tags(doc):
    paper_tag = doc.find_all("a", {"class": "gsc_a_at"})
    cite_tag = doc.select('[title=Cite] + a')
    link_tag = doc.find_all('h3',{"class" : "gs_rt"})
    author_tag = doc.find_all("div", {"class": "gs_gray"})
    return paper_tag,cite_tag,link_tag,author_tag

In [258]:
# it will return the title of the paper
def get_papertitle(paper_tag):
    paper_names = []
    for tag in paper_tag:
        paper_names.append(tag.select('h3')[0].get_text())
    return paper_names

In [259]:
def get_article(author_tag,paper_tag):
    journal_names=[]
    journal_pages=[]
    pub_years=[]
    papers=[]
    for i in range(len(author_tag)):
    #     if i%2==0:
    #         print()
        if i%2==1:
            pub_info=author_tag[i].text.split(',')
            journal_name=pub_info[0].strip()
            pub_year=pub_info[-1].strip()
            if len(pub_info)==3:
                journal_page=pub_info[-2].strip()
            else:
                journal_page=''
            journal_names.append(journal_name)
            journal_pages.append(journal_page)
            pub_years.append(pub_year)

    for idx in range(len(paper_tag)):
        papers.append(paper_tag[idx].text.strip())
    return journal_names,journal_pages,pub_years,papers

In [260]:
paper_doc=get_paperinfo(url)
paper_tag,cite_tag,link_tag,author_tag=get_tags(paper_doc)

In [261]:
journal_names,journal_pages,pub_years,papers=get_article(author_tag,paper_tag)

In [265]:
jourNames=[]
jourPages=[]
pubYears=[]
Papers=[]
for i in range (0,100,10):
    # get url for the each page
    url = "https://scholar.google.com/citations?hl=en&user=M_BRU-4AAAAJ".format(i)
#     print(url)
    # function for the get content of each page
    doc = get_paperinfo(url)
    
    # function for the collecting tags
    paper_tag,cite_tag,link_tag,author_tag = get_tags(doc)
    
    journal_names,journal_pages,pub_years,papers=get_article(author_tag,paper_tag)
#     print(papers)
#     print()
    jourNames.extend(journal_names)
    jourPages.extend(journal_pages)
    pubYears.extend(pub_years)
    Papers.extend(papers)
    
    # use sleep to avoid status code 429
#     time.sleep(30)

In [9]:
with open('data/Xing_SCIs.txt','r',encoding="utf8") as file:
    a=file.readlines()
#     print(a)

In [16]:
articles=[]
for i in range(len(a)):
    if i%7==0:
        tmp=a[i:i+7]
        tmp=[t.strip() for t in tmp]
        articles.append(tmp)
#         print(tmp)
#     print(a[i])
#     print()
#     print(a[i])

In [18]:
with open('xingArticlesFull_updated.json', 'w') as outfile:
    json.dump(articles, outfile)

In [35]:
with open('data/bookChapters.txt','r',encoding="utf8") as file:
    a=file.readlines()

In [36]:
posters=[]
for i in range(len(a)):
    if i%4==0:
        tmp=a[i:i+4]
        tmp=[t.strip() for t in tmp]
        posters.append(tmp)

In [37]:
with open('posters.json', 'w') as outfile:
    json.dump(posters, outfile)

In [38]:
posters

[['Guo, Y., Xing, W., & H., Lee (in press).',
  'Identifying students’ mechanistic explanations in textual responses to science questions with association rule mining.',
  'In Proceedings of the 2015 International Conference on Data Mining (ICDM). Atlantic City, NJ: IEEE',
  'pub/IEEE_ICDM_draft.pdf'],
 ['Xing, W., Kim, S. & Goggins, S. (2015).',
  'Modeling performance in asynchronous CSCL: An exploration of social ability, collective efficacy and social interaction.',
  'Exploring the Material Conditions of Learning: Proceedings of The Computer Supported Collaborative Learning (CSCL 2015), (Vol, pp. 276-283). Gothenburg, Sweden: International Society of the Learning Sciences.',
  'pub/CSCL_2015_xing.pdf'],
 ['Xing, W., & Goggins, S. (2015).',
  'Learning analytics in outer space:a Hidden Naive Bayes model for automatic student off-task behavior detection.',
  "In Proceedings of the Fifth International Conference on Learning Analytics And Knowledge - LAK'15 (pp. 176-183). New York, NY

In [23]:
a

['Guo, Y., Xing, W., & H., Lee (in press).\n',
 'Identifying students’ mechanistic explanations in textual responses to science questions with association rule mining.\n',
 'In Proceedings of the 2015 International Conference on Data Mining (ICDM). Atlantic City, NJ: IEEE\n',
 '\n',
 'Xing, W., Kim, S. & Goggins, S. (2015).\n',
 'Modeling performance in asynchronous CSCL: An exploration of social ability, collective efficacy and social interaction.\n',
 'Exploring the Material Conditions of Learning: Proceedings of The Computer Supported Collaborative Learning (CSCL 2015), (Vol, pp. 276-283). Gothenburg, Sweden: International Society of the Learning Sciences. \n',
 '\n',
 'Xing, W., & Goggins, S. (2015).\n',
 'Learning analytics in outer space:a Hidden Naive Bayes model for automatic student off-task behavior detection.\n',
 "In Proceedings of the Fifth International Conference on Learning Analytics And Knowledge - LAK'15 (pp. 176-183). New York, NY, USA: ACM.\n",
 '10.1145/2723576.272